In [13]:
import pandas as pd
import glob
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StringType,BooleanType,DateType,IntegerType,FloatType
spark = SparkSession.builder.appName("OTR").config("spark.sql.caseSensitive", "True").getOrCreate()



In [25]:
df = spark.read.option("header",True) \
     .csv("./data/output/Master Ledger.xlsx - Master_Ledger.csv")
df = df.withColumn("ID",col("ID").cast(IntegerType()))\
        .withColumn("Amount",col("Amount").cast(FloatType()))\
        .withColumn("Subscriptions",col("Subscriptions").cast(BooleanType()))\
        .withColumn("Return",col("Return").cast(BooleanType()))
df = df.select(col("Date"),to_date(col("Date"),"MM/dd/yyyy").alias("Date"))
df.printSchema()


df.orderBy("ID", ascending=False).show()


root
 |-- Date: string (nullable = true)
 |-- Date: date (nullable = true)



Py4JJavaError: An error occurred while calling o819.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 34.0 failed 1 times, most recent failure: Lost task 0.0 in stage 34.0 (TID 34) (192.168.0.26 executor driver): org.apache.spark.SparkUpgradeException: [INCONSISTENT_BEHAVIOR_CROSS_VERSION.PARSE_DATETIME_BY_NEW_PARSER] You may get a different result due to the upgrading to Spark >= 3.0:
Fail to parse '3/24/2021' in the new parser. You can set "spark.sql.legacy.timeParserPolicy" to "LEGACY" to restore the behavior before Spark 3.0, or set to "CORRECTED" and treat it as an invalid datetime string.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failToParseDateTimeInNewParserError(QueryExecutionErrors.scala:1368)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:149)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$

In [ ]:
csv_files = glob.glob('./data/*.csv')
csv_files

['./data\\2023-02-08 thru 2023-05-08 transactions.csv']

In [ ]:
df_list = (pd.read_csv(file) for file in csv_files)
df  = pd.concat(df_list, ignore_index=True)
df

,Date,Account,Description,Category,Tags,Amount
0,2023-05-07,Chase FFU PA 7013,Automatic Payment - Thank,Credit Card Payments,NaN,100.00
1,2023-05-06,Discover It PA 1762,"Sq *ichiran Usa., Inc. Brooklyn Ny Xxxxxxxxxxx...",Restaurants,NaN,-56.62
2,2023-05-06,Discover It PA 1762,Instant Ink,Printing,NaN,-2.11
3,2023-05-06,Unlimited Cash Ritu 3035,Payment - Thank You,Credit Card Payments,NaN,35.00
4,2023-05-06,Amazon Prime Ritu 9122,Prime Video*9h56y4tr3,Entertainment,NaN,-8.28
...,...,...,...,...,...,...
679,2023-02-09,Adv Plus Banking Ritu 9759,Klarna,Loans,NaN,-21.52
680,2023-02-09,Adv Plus Banking Ritu 9759,Yifang Taiwan Fruit Tea,Restaurants,NaN,-17.97
681,2023-02-09,Adv Plus Banking Ritu 9759,Klarna,Loans,NaN,-8.23
682,2023-02-08,Citi / Aadvantage Platinum Ritu 9546,Charleskeith.com Pte L Singapore Sgp,Refunds & Reimbursements,NaN,77.40
